In [ ]:
import json

import hvplot.xarray  # noqa F401
import pystac
import s3fs
from odc.stac import configure_rio, load

In [ ]:
import odc.stac
if odc.stac.__version__ != "0.3.6":
    raise Exception(f"You need to use odc.stac version 0.3.6 or greater. You have {odc.stac.__version__}")

In [ ]:
s3_uri = "s3://files.auspatious.com/hsi_example/TD1_004930_20230205_L2A_20230224_03001065_COG.stac-item.json"

# Open S3 object as a file using s3fs
s3 = s3fs.S3FileSystem(anon=True)
with s3.open(s3_uri, "rt") as f:
    stac_dict = json.load(f)
item = pystac.read_dict(stac_dict)

# Optionally select a subset, so it doesn't take a long time
eo_bands_subset = item.assets["reflectance"].extra_fields["eo:bands"]

# Load the data, telling rasterio to not sign requests
configure_rio(
    cloud_defaults=True,
    aws={"aws_unsigned": True},
    AWS_S3_ENDPOINT="s3.ap-southeast-2.amazonaws.com",
)
ds = load(
    [item],
    measurements=[i["name"] for i in eo_bands_subset],
    chunks={"bands": 1, "longitude": 1200, "latitude": 1200}
)

# No need for time
ds = ds.squeeze("time")

# Stack up the bands, so we have a multi-dimensional raster instead
ds_stacked = ds.to_array("bands")

# Replace the original ds object with a nice indexed one
bands = list([float(i["description"]) for i in eo_bands_subset])
bands.sort()  # This is pretty dangerous... let's assume the .tif has bands in the right order!
ds = ds_stacked.assign_coords(bands=bands).to_dataset(name="reflectance")

# This is a dask lazy-loaded array. Can be all loaded with .compute()
ds


In [ ]:
# Now we can select a band by wavelength and plot it
ds.reflectance.sel(bands=450, method="nearest").hvplot(aspect="equal", frame_width=600)

In [ ]:
# Create a water layer
# Picked values from here https://en.wikipedia.org/wiki/Normalized_difference_water_index
green = ds.reflectance.sel(bands=559, method="nearest").astype("float32")
nir = ds.reflectance.sel(bands=864, method="nearest").astype("float32")

water = ((green - nir) / (green + nir)) > 0.2
ds["water"] = water.fillna(float("nan")).where(water)

In [ ]:
ds.water.hvplot(aspect="equal", frame_width=600)